In [1]:
import copy
import math
import matplotlib.pyplot as plt
import numpy as np
from random import shuffle
from heapq import heappop, heappush, heapify
from random import randint
import time
from sys import float_info
EPS = float_info.epsilon
%matplotlib notebook
%load_ext autoreload
%autoreload 2

### Search Node Representation

In [2]:
class Node:
    def __init__(self, coords, g = 0, h = 0, F = None, parent = None, f=None):
        self.coords = coords # tuple of coords
        self.g = g
        self.h = h
        if F is None:
            self.F = self.g + h
        else:
            self.F = F 
        if f is None:
            self.f = self.g + h
        else:
            self.f = f
        self.parent = parent
        self.k = 0
    
    
    def __eq__(self, other):
        return self.coords == other.coords
    
    def __lt__(self, other):
        return self.F < other.F or ((self.F == other.F) and (self.h < other.h)) \
               or ((self.F == other.F) and (self.h == other.h) and (self.k > other.k))
    
    def __hash__(self): 
        return hash(self.coords)

In [3]:
import heapq
import itertools

REMOVED = Node((-1,-1), -1)

class Open:
    def __init__(self):
        self.elements = []
        self.count = itertools.count()
        self.entry_finder = {}
        self.node_finder = {}
    
    def __iter__(self):
        for entry in self.elements:
            if entry[2] is not REMOVED:
                  yield entry[2]
                    
    def __len__(self):
        return len(self.elements)

    def isEmpty(self):
        return (len(self.elements) == 0)               
    
    def AddNode(self, node : Node, *args):
        if (node.coords, node.g) in self.entry_finder:
            if self.entry_finder[(node.coords, node.g)][2].F < node.F:
                return
            t = self.entry_finder.pop((node.coords, node.g))
            t[2] = REMOVED
        entry = [node.F, next(self.count), node]
        self.entry_finder[(node.coords, node.g)] = entry
        self.node_finder[node.coords] = node.g
        heapq.heappush(self.elements, entry)

    def GetBestNode(self, *args):
        while self.elements:
            entry = heapq.heappop(self.elements)
            if entry[2] is not REMOVED:
                del self.entry_finder[(entry[2].coords, entry[2].g)]
                if entry[2].coords in self.node_finder:
                    del self.node_finder[entry[2].coords]
                return entry[2]
            
    def InOpen(self, item: Node):
        return (item.coords in self.node_finder)
    
    def GetGValue(self, item: Node):
        return self.node_finder[item.coords]

In [4]:
class OpenList:
    def __init__(self):
        self.elements = []
        self.node_finder = set()
    

    def __iter__(self):
        return iter(self.elements)


    def __len__(self):
        return len(self.elements)


    def isEmpty(self):
        if len(self.elements) != 0:
            return False
        return True
    

    def GetBestNode(self, *args):
        bestF = math.inf
        bestCoord = 0
        for i in range(len(self.elements)):
            if self.elements[i].F < bestF:
                bestCoord = i
                bestF = self.elements[i].F
                
        best = self.elements.pop(bestCoord)
        self.node_finder.remove(best.coords)
        return best
    

    def AddNode(self, node : Node, *args):
        for existingNode in self.elements:
            if existingNode.coords == node.coords:
                if existingNode.F > node.F:
                    existingNode.g = node.g
                    existingNode.F = node.F
                    existingNode.parent = node.parent
                    return
                else:
                    return
        self.elements.append(node)
        self.node_finder.add(node.coords)
        return
    
    def InOpen(self, item: Node):
        return (item.coords in self.node_finder)

In [5]:
class Closed:
    def __init__(self):
        self.elements = {}

    def __iter__(self):
        return iter(self.elements)
    
    def __len__(self):
        return len(self.elements)

    def AddNode(self, item : Node):
        self.elements[item] = item.g

    def WasExpanded(self, item : Node):
        return (item in self.elements)
    
    def DeleteNode(self, item: Node):
        del self.elements[item]

In [6]:
import utils

pam250 = utils.load_pam250_matrix()

# Heuristic function

In [7]:
def hfunc(coords, distances2d):
    h = 0
    for i, coord1 in enumerate(coords):
        for j, coord2 in enumerate(coords):
            if i <= j:
                continue
            h += distances2d[(i, j)][-coord1 - 1, -coord2 - 1]
    return h          

In [8]:
def get_neighbors(coords, seqs):
    num_coords = len(coords)
    t = np.arange(1, 2 ** num_coords, dtype=np.uint8)
    t = np.unpackbits(t).reshape(-1, 8)
    max_coords = list(map(len, seqs))
    t = coords + t[:, -num_coords:]
    return t[np.all(t <= max_coords, axis=1)]

In [9]:
def compute_cost(coords1, coords2, seqs, pam250):
    d = 0
    for i, _ in enumerate(coords2):
        for j, _ in enumerate(coords2):
            if i <= j:
                continue
            if coords1[i] == coords2[i] and coords1[j] == coords2[j]:
                continue
            if coords1[i] == coords2[i] or coords1[j] == coords2[j]:
                d += 8
            else:
                d += pam250[seqs[i][coords1[i]], seqs[j][coords1[j]]]
    return d

In [10]:
def astar(seqs):
    OPEN = Open()
    CLOSED = Closed()
    distances2d = utils.get_distances2d(seqs, pam250)
    
    num_seqs = len(seqs)
    start_coords = tuple(0 for i in range(num_seqs))
    end_coords = tuple(len(seq) for seq in seqs)
    
    start_node = Node(start_coords, 0, hfunc(start_coords, distances2d))
    end_node = Node(end_coords)
    OPEN.AddNode(start_node)
    max_n = 0
    
    while not OPEN.isEmpty():
        s = OPEN.GetBestNode(CLOSED)
        CLOSED.AddNode(s)
        if s == end_node:
            cum_n = len(CLOSED)
            return (True, s, CLOSED, OPEN, max_n, cum_n)
        neighbors = get_neighbors(s.coords, seqs)
        for neighbor in neighbors:
            neighbor = tuple(neighbor)
            if not CLOSED.WasExpanded(Node(neighbor)):
                node = Node(neighbor, s.g + compute_cost(s.coords, neighbor, seqs, pam250),
                            hfunc(neighbor, distances2d))
                node.parent = s
                OPEN.AddNode(node)
        max_n = max(max_n, len(CLOSED) + len(OPEN.node_finder))
    return False, None, CLOSED, OPEN

In [11]:
def peastar(seqs, cutoff):
    OPEN = Open()
    CLOSED = Closed()
    distances2d = utils.get_distances2d(seqs, pam250)
    
    cumulative_nodes = 0
    maximum_nodes = 0
    
    num_seqs = len(seqs)
    start_coords = tuple(0 for i in range(num_seqs))
    end_coords = tuple(len(seq) for seq in seqs)
    
    start_node = Node(start_coords, 0, hfunc(start_coords, distances2d))
    end_node = Node(end_coords)
    OPEN.AddNode(start_node)
    
    g_values = dict()
    g_values[start_coords] = 0
    
    while not OPEN.isEmpty():
        s = OPEN.GetBestNode()
        
        if s == end_node:
            return (True, s, CLOSED, OPEN, maximum_nodes, cumulative_nodes)
        neighbors = get_neighbors(s.coords, seqs)
        min_f_n = None
        cumulative_nodes += 1
        for neighbor in neighbors:
            neighbor = tuple(neighbor)
            n = Node(neighbor)
            
            g_n = math.inf
            if neighbor in g_values:
                g_n = min(g_values[neighbor], s.g + compute_cost(s.coords, neighbor, seqs, pam250))
            else:
                g_n = s.g + compute_cost(s.coords, neighbor, seqs, pam250)
                g_values[neighbor] = math.inf
            h_n = hfunc(neighbor, distances2d)
            f_n = g_n + h_n
            if f_n <= s.F + cutoff:
                if (CLOSED.WasExpanded(n) and 
                    s.g + compute_cost(s.coords, neighbor, seqs, pam250) < g_values[neighbor]):
                    g_n = s.g + compute_cost(s.coords, neighbor, seqs, pam250)
                    n = Node(neighbor, g_n, h_n)
                    n.parent = s
                    CLOSED.DeleteNode(n)
                    OPEN.AddNode(n)
                    
                    g_values[neighbor] = g_n
                elif (OPEN.InOpen(n) and
                    s.g + compute_cost(s.coords, neighbor, seqs, pam250) < g_values[neighbor]):
                    g_n = s.g + compute_cost(s.coords, neighbor, seqs, pam250)
                    n = Node(neighbor, g_n, h_n)
                    n.parent = s
                    OPEN.AddNode(n)
                    
                    g_values[neighbor] = g_n
                elif (not OPEN.InOpen(n)) and (not CLOSED.WasExpanded(n)):
                    g_n = s.g + compute_cost(s.coords, neighbor, seqs, pam250)
                    n = Node(neighbor, g_n, h_n)
                    n.parent = s
                    OPEN.AddNode(n)
                    
                    if neighbor in g_values:
                        g_values[neighbor] = min(g_n, g_values[neighbor])
                    else:
                        g_values[neighbor] = g_n
            else:
                if min_f_n is None:
                    min_f_n = f_n
                else:
                    min_f_n = min(f_n, min_f_n)
            
        if min_f_n is None:
            CLOSED.AddNode(s)
        else:
            s.F = min_f_n
            OPEN.AddNode(s) 
        maximum_nodes = max(maximum_nodes, len(CLOSED) + len(OPEN.node_finder))

# Tests

In [12]:
def alignment_astar(seqs):
    _, s, _, _, max_n, cum_n = astar(seqs)
    print(f'Stored Nodes = {max_n}, Cumulative Expansion = {cum_n}')
    return utils.make_alignment(s, seqs)
def alignment_peastar(seqs, c=3):
    _, s, _, _, max_n, cum_n = peastar(seqs, c)
    print(f'Stored Nodes = {max_n}, Cumulative Expansion = {cum_n}')
    return utils.make_alignment(s, seqs)

In [13]:
seqs = ['AA', 'AA']
assert(np.all(alignment_astar(seqs) == utils.get_alignment(seqs[0], seqs[1], pam250)))
seqs = ['CGT', 'CT']
assert(np.all(alignment_astar(seqs) == utils.get_alignment(seqs[0], seqs[1], pam250)))

Stored Nodes = 7, Cumulative Expansion = 3
Stored Nodes = 9, Cumulative Expansion = 4


In [14]:
seqs = ['ATAT', 'TA', 'CTA', 'RTV', 'TV', 'ATCT']

In [15]:
_, s, c, o, _, _ = astar(seqs)
print(utils.make_alignment(s, seqs))
print(len(o), len(c))

[['A' 'T' 'A' 'T']
 ['_' 'T' '_' 'A']
 ['C' 'T' '_' 'A']
 ['R' 'T' '_' 'V']
 ['_' 'T' '_' 'V']
 ['A' 'T' 'C' 'T']]
248 5


In [16]:
_, s, c, o, _, _ = peastar(seqs, 0)
print(utils.make_alignment(s, seqs))
print(len(o), len(c))

[['A' 'T' 'A' 'T']
 ['_' 'T' '_' 'A']
 ['C' 'T' '_' 'A']
 ['R' 'T' '_' 'V']
 ['_' 'T' '_' 'V']
 ['A' 'T' 'C' 'T']]
4 0


# Tests with real data

In [17]:
import os

def load_seqs(path):
    species = []
    seqs = []
    for file in os.listdir(path):
        with open(f'{path}/{file}', 'r') as f:
            species.append(file[:-4]) 
            seqs.append(f.read().replace('\n', ''))
    return species, seqs

species, seqs = load_seqs('data')

In [18]:
%%time
utils.calc_alignment_score(alignment_astar(seqs[6:9]), pam250)

Stored Nodes = 3566, Cumulative Expansion = 627
Wall time: 4.73 s


-3463

In [19]:
%%time
utils.calc_alignment_score(alignment_peastar(seqs[6:9], 5), pam250)

Stored Nodes = 655, Cumulative Expansion = 699
Wall time: 4.52 s


-3463

# Alignment of 7 sequences

Test 1

In [20]:
%%time
utils.calc_alignment_score(alignment_peastar(seqs[:7], 0), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[:7], 10), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[:7], 50), pam250)
utils.calc_alignment_score(alignment_astar(seqs[:7]), pam250)

Stored Nodes = 7502, Cumulative Expansion = 13533
Stored Nodes = 8280, Cumulative Expansion = 10854
Stored Nodes = 15937, Cumulative Expansion = 9866
Stored Nodes = 305929, Cumulative Expansion = 7502
Wall time: 6min 44s


-34581

Test 2

In [21]:
%%time
utils.calc_alignment_score(alignment_peastar(seqs[1:8], 0), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[1:8], 10), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[1:8], 50), pam250)
utils.calc_alignment_score(alignment_astar(seqs[1:8]), pam250)

Stored Nodes = 8832, Cumulative Expansion = 16115
Stored Nodes = 9695, Cumulative Expansion = 13128
Stored Nodes = 19333, Cumulative Expansion = 11033
Stored Nodes = 363769, Cumulative Expansion = 8832
Wall time: 7min 39s


-33581

Test 3

In [22]:
indices = np.arange(len(seqs))
shuffle(indices)
indices = indices[:7]

In [23]:
indices

array([19, 13, 14,  0, 17, 12, 15])

In [24]:
seqs = np.array(seqs)

In [25]:
%%time
utils.calc_alignment_score(alignment_peastar(seqs[indices], 0), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[indices], 10), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[indices], 50), pam250)
utils.calc_alignment_score(alignment_astar(seqs[indices]), pam250)

Stored Nodes = 60678, Cumulative Expansion = 134837
Stored Nodes = 66062, Cumulative Expansion = 111608
Stored Nodes = 124324, Cumulative Expansion = 88005
Stored Nodes = 1209639, Cumulative Expansion = 60678
Wall time: 2h 5min 47s


-27065

Test 4

In [26]:
indices = np.arange(len(seqs))
shuffle(indices)
indices = indices[:7]

In [27]:
indices

array([ 8, 12,  3,  5, 17,  9,  2])

In [28]:
%%time
utils.calc_alignment_score(alignment_peastar(seqs[indices], 0), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[indices], 10), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[indices], 50), pam250)
utils.calc_alignment_score(alignment_astar(seqs[indices]), pam250)

Stored Nodes = 11639, Cumulative Expansion = 21270
Stored Nodes = 12691, Cumulative Expansion = 17901
Stored Nodes = 24469, Cumulative Expansion = 15563
Stored Nodes = 437336, Cumulative Expansion = 11638
Wall time: 12min 17s


-29118

Test 5

In [29]:
indices = np.arange(len(seqs))
shuffle(indices)
indices = indices[:7]

In [30]:
indices

array([ 7,  2, 13,  0,  3, 17,  5])

In [31]:
%%time
utils.calc_alignment_score(alignment_peastar(seqs[indices], 0), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[indices], 10), pam250)
utils.calc_alignment_score(alignment_peastar(seqs[indices], 50), pam250)
utils.calc_alignment_score(alignment_astar(seqs[indices]), pam250)

Stored Nodes = 41864, Cumulative Expansion = 95638
Stored Nodes = 46361, Cumulative Expansion = 77945
Stored Nodes = 88605, Cumulative Expansion = 69407
Stored Nodes = 786307, Cumulative Expansion = 41864
Wall time: 1h 7min 8s


-31485